In [4]:
# Import necessary PyTorch libraries
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
 
# Additional libraries for visualization and utilities
import matplotlib.pyplot as plt
import numpy as np

In [5]:
def get_device():
    """Selects the best available device for PyTorch computations.

    Returns:
        torch.device: The selected device.
    """

    if torch.cuda.is_available():
        return torch.device('cuda')
    elif torch.backends.mps.is_available():
        return torch.device('mps')
    else:
        return torch.device('cpu')

device = get_device()
print(f"using device: {device}")

using device: cuda


In [6]:
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import Compose, ToTensor, Normalize, Grayscale

# Define transformations: Convert to grayscale, resize if needed, and normalize the data
transform = Compose([
    Grayscale(num_output_channels=1),  # Convert to grayscale
    ToTensor(),
    Normalize((0.5,), (0.5,))  # Normalize with single channel
])

# Load the CIFAR-10 dataset
dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Splitting dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=False)

print("Data loaders created for training and validation.")

Files already downloaded and verified
Data loaders created for training and validation.


In [7]:
def get_timestep_embedding(timesteps, embedding_dim):
    """
    This matches the implementation in Denoising Diffusion Probabilistic Models:
    From Fairseq.
    Build sinusoidal embeddings.
    This matches the implementation in tensor2tensor, but differs slightly
    from the description in Section 3.5 of "Attention Is All You Need".
    """
    assert len(timesteps.shape) == 1

    half_dim = embedding_dim // 2
    emb = math.log(10000) / (half_dim - 1)
    emb = torch.exp(torch.arange(half_dim, dtype=torch.float32) * -emb)
    emb = emb.to(device=timesteps.device)
    emb = timesteps.float()[:, None] * emb[None, :]
    emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=1)
    if embedding_dim % 2 == 1:  # zero pad
        emb = torch.nn.functional.pad(emb, (0,1,0,0))
    return emb

In [8]:
class Decoder(nn.Module):
    def __init__(self, latent_dims, output_shape, timestep_dim=128):
        super(Decoder, self).__init__()
        self.latent_dims = latent_dims
        self.output_shape = output_shape
        self.timestep_dim = timestep_dim

        self.conv1 = nn.Conv2d(1, latent_dims[0], kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(latent_dims[0], latent_dims[1], kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(latent_dims[1], latent_dims[2], kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(latent_dims[2], latent_dims[3], kernel_size=3, stride=2, padding=1)
        self.conv5 = nn.Conv2d(latent_dims[3], latent_dims[4], kernel_size=3, stride=1, padding=1)

        self.timestep_mlp = nn.Sequential(
            nn.Linear(timestep_dim, latent_dims[4]),
            nn.ReLU(),
            nn.Linear(latent_dims[4], latent_dims[4]),
        )

        self.deconv1 = nn.ConvTranspose2d(latent_dims[4] * 2, latent_dims[3], kernel_size=3, stride=1, padding=1)
        self.deconv2 = nn.ConvTranspose2d(latent_dims[3], latent_dims[2], kernel_size=4, stride=2, padding=1)
        self.deconv3 = nn.ConvTranspose2d(latent_dims[2], latent_dims[1], kernel_size=3, stride=1, padding=1)
        self.deconv4 = nn.ConvTranspose2d(latent_dims[1], latent_dims[0], kernel_size=4, stride=2, padding=1)
        self.deconv5 = nn.ConvTranspose2d(latent_dims[0], output_shape[0], kernel_size=3, stride=1, padding=1)

    def forward(self, x, t):
        timestep_emb = get_timestep_embedding(t, self.timestep_dim)
        timestep_emb = self.timestep_mlp(timestep_emb)

        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = torch.relu(self.conv4(x))
        x = torch.relu(self.conv5(x))

        x = torch.cat([x, timestep_emb[:, :, None, None].repeat(1, 1, x.shape[2], x.shape[3])], dim=1)

        x = torch.relu(self.deconv1(x))
        x = torch.relu(self.deconv2(x))
        x = torch.relu(self.deconv3(x))
        x = torch.relu(self.deconv4(x))
        x = torch.sigmoid(self.deconv5(x))

        return x

In [9]:
class EchoModel(nn.Module):
    def __init__(self, input_shape, latent_dims, output_shape, T=1000, batch_size=100):
        super(EchoModel, self).__init__()
        self.input_shape = input_shape
        self.latent_dims = latent_dims
        self.output_shape = output_shape
        self.T = T
        self.batch_size = batch_size

        # self.encoder = Encoder(input_shape, latent_dims)
        self.decoder = Decoder(latent_dims, output_shape)

        # Define the noise schedule
        self.alpha = self.create_noise_schedule(T)

    def create_noise_schedule(self, T):
        alpha = torch.linspace(0.9999, 1e-5, T)
        return alpha

    def forward(self, x):

        # Calculate z as a gaussian noise tensor
        z = torch.randn(self.batch_size,1, 32, 32).to(device)
        
        torch.cuda.empty_cache()
        
        # Retrieve noise scheduler alpha_T
        alpha_T = self.alpha[-1]

        # Calculate square root alphas
        sqrt_alpha_T = torch.sqrt(alpha_T)
        sqrt_one_minus_alpha_T = torch.sqrt(1 - alpha_T)
        
        # Perform the weighted sum
        x_T = sqrt_alpha_T * x + sqrt_one_minus_alpha_T * z

        del z
        torch.cuda.empty_cache()


        # Perform the reconstruction process using Algorithm 2
        reconstructed_x = self.reconstruct(x_T)
        torch.cuda.empty_cache()
        return reconstructed_x

    def reconstruct(self, x_T):
        x_s = x_T
        x_0_hat = x_T
        for s in range(self.T-1, 0, -1):
            t = torch.tensor([s] * x_T.size(0), dtype=torch.long).to(x_T.device)
            sqrt_alpha_s = torch.sqrt(self.alpha[s])
            sqrt_one_minus_alpha_s = torch.sqrt(1 - self.alpha[s])

            # Estimate the original image using the decoder
            x_0_hat = self.decoder(x_s, t)

            # Calculate the estimated noise using Eq. (3)
            z_hat = (x_s - sqrt_alpha_s * x_0_hat) / sqrt_one_minus_alpha_s

            # Calculate D(x_0_hat, s) and D(x_0_hat, s-1) using Eq. (5) and (6)
            D_x_0_hat_s = sqrt_alpha_s * x_0_hat + sqrt_one_minus_alpha_s * z_hat
            D_x_0_hat_s_minus_1 = torch.sqrt(self.alpha[s-1]) * x_0_hat + torch.sqrt(1 - self.alpha[s-1]) * z_hat

            # Update x_s using Eq. (7)
            x_s = x_s - D_x_0_hat_s + D_x_0_hat_s_minus_1

        return x_0_hat

In [10]:
from torch.cuda.amp import GradScaler, autocast
import time  # Importing time to log the duration



def train(model, optimizer, train_loader, device, num_epochs, accumulation_steps=2, checkpoint_path="checkpoint.pth"):
    model.train()
    scaler = GradScaler()  # Initialize the gradient scaler

    for epoch in range(num_epochs):
        epoch_loss = 0.0
        # optimizer.zero_grad()  # Move optimizer.zero_grad() outside the batch loop for gradient accumulation
        epoch_start_time = time.time()  # Time tracking for the epoch

        print(f"Starting epoch {epoch+1}/{num_epochs}")
        for batch_idx, (data, _) in enumerate(train_loader):
            batch_start_time = time.time()  # Time tracking for the batch
            data = data.to(device)

            with autocast():  # Enable automatic mixed precision
                reconstructed_x = model(data)
                total_loss = nn.functional.mse_loss(reconstructed_x, data)
                print(f"total loss: {total_loss}")
            # Log before the backward pass
            print(f"\tBatch {batch_idx+1}/{len(train_loader)}, Forward pass done, starting backward pass.")

            # Scale the loss, but don't call optimizer.step() yet
            if not torch.isnan(total_loss).any():
                scaler.scale(total_loss).backward()
            else:
                print(f"Warning: NaN detected in total_loss at batch {batch_idx+1}, skipping backward pass.")


            if (batch_idx + 1) % accumulation_steps == 0 or (batch_idx + 1) == len(train_loader):
                scaler.step(optimizer)  # Only step the optimizer every `accumulation_steps`
                scaler.update()
                optimizer.zero_grad()  # Reset gradients only after accumulation

            print(f"total loss item: {total_loss.item()}")

            # Safe-guarding against NaN for epoch_loss
            if not torch.isnan(total_loss).any():
                epoch_loss += total_loss.item()
            else:
                print(f"NaN detected, not adding to epoch_loss at batch {batch_idx+1}")
            print(f"Epoch loss: {epoch_loss}")

            # Log after a batch is processed
            print(f"\tBatch {batch_idx+1}/{len(train_loader)} processed in {time.time() - batch_start_time:.2f} seconds.")

        # Average loss after training for an epoch
        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}] completed in {time.time() - epoch_start_time:.2f} seconds, Avg Loss: {avg_loss}")

        # Validation phase
        avg_val_loss = validate(model, val_loader, device)
        print(f"Epoch [{epoch+1}/{num_epochs}] validation completed, Avg Validation Loss: {avg_val_loss}")
        

    return model

In [11]:
# Define the input shape, latent dimensions, and output shape
input_shape = (1, 32, 32)  # Shape for grayscale CIFAR-10 (1 channel, 32x32 images)
latent_dims = [32, 64, 128, 256, 512]  # Updated latent dimensions
output_shape = (1, 32, 32)  # Shape for grayscale CIFAR-10

# Create an instance of the EchoModel
model = EchoModel(input_shape, latent_dims, output_shape).to(device)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Define the number of epochs and loss weights
num_epochs = 100

# Train the model
trained_model = train(model, optimizer, train_loader, device, num_epochs)

Starting epoch 1/100
total loss: 0.5047832131385803
	Batch 1/400, Forward pass done, starting backward pass.
total loss item: 0.5047832131385803
Epoch loss: 0.5047832131385803
	Batch 1/400 processed in 4.62 seconds.
total loss: 0.5407262444496155
	Batch 2/400, Forward pass done, starting backward pass.
total loss item: 0.5407262444496155
Epoch loss: 1.0455094575881958
	Batch 2/400 processed in 5.68 seconds.
total loss: 0.4463938772678375
	Batch 3/400, Forward pass done, starting backward pass.
total loss item: 0.4463938772678375
Epoch loss: 1.4919033348560333
	Batch 3/400 processed in 5.58 seconds.
total loss: 0.511258065700531
	Batch 4/400, Forward pass done, starting backward pass.
total loss item: 0.511258065700531
Epoch loss: 2.0031614005565643
	Batch 4/400 processed in 5.69 seconds.
total loss: 0.33101943135261536
	Batch 5/400, Forward pass done, starting backward pass.
total loss item: 0.33101943135261536
Epoch loss: 2.3341808319091797
	Batch 5/400 processed in 5.61 seconds.
tota

ValueError: too many values to unpack (expected 3)

In [12]:
def validate(model, val_loader, device):
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0
    with torch.no_grad():  # Disable gradient computation during validation
        for data, _ in val_loader:
            data = data.to(device)
            reconstructed_x = model(data)
            reconstruction_loss = nn.functional.mse_loss(reconstructed_x, data)
            total_val_loss += reconstruction_loss.item()  # Accumulate the validation loss
    
    avg_val_loss = total_val_loss / len(val_loader)  # Calculate average loss
    return avg_val_loss

# Validation phase
avg_val_loss = validate(model, val_loader, device)

NameError: name 'epoch' is not defined

In [ ]:

print(f"Epoch [{epoch+1}/{num_epochs}] validation completed, Avg Validation Loss: {avg_val_loss}")

In [22]:
import os

def save_checkpoint(epoch, model, optimizer, filename="checkpoint.pth"):
    """Saves the model and optimizer state at the specified path."""
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, filename)
    print(f"Checkpoint saved at epoch {epoch} to {filename}")


Checkpoint saved at epoch 1 to checkpoint-gaussian.pth
